# 0. Instalacion de utilitarios Pyspark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.3.4/spark-2.3.4-bin-hadoop2.7.tgz
!tar xf spark-2.3.4-bin-hadoop2.7.tgz

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


In [ ]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 53kB/s 
     |████████████████████████████████| 204kB 42.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=a5057c1fd71f5a4945c3797b45a38dda1c2a1dd9d4bc590a6a3dfb7eef25ea42
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

# 1. Lectura del origen datos

In [ ]:
# Lectura desde un archivo fisico
# from google.colab import files
# uploaded = files.upload()

In [ ]:

# Conexion al drive
#---------------------------
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
ls

drive/        spark-2.3.4-bin-hadoop2.7/     spark-2.4.4-bin-hadoop2.7/
sample_data/  spark-2.3.4-bin-hadoop2.7.tgz  spark-2.4.4-bin-hadoop2.7.tgz


#2.  Cargando los datos en formato Spark

In [ ]:
DS_spam = spark.read.csv('/content/drive/My Drive/PySpark Machine Learning en plataforma Big Data/Training R ML en Casos de Negocios/9_Analisis_texto_Spam_emaling/Identificando_correos_spam_vf.csv', sep=';', header=True, inferSchema=True)

# 2.1 Revision de formatos 
DS_spam.printSchema()

In [ ]:
# Dimensiona de base
print((DS_spam.count(),len(DS_spam.columns)))

In [ ]:
# 2.2 Revisiones  
DS_spam.show(5, truncate = False)

In [ ]:
# 2.3 Validacion de carga del total de registros
print(DS_spam.count())

In [ ]:
# 2.4 Exploracion inicial
DS_spam.describe().show()

In [ ]:
# 2.5 Los casos de Spam
DS_spam.groupBy('clase').count().show(10,False)

In [ ]:
#2.6 Filtrando los casos a analizar 
DS_spam_2=DS_spam.filter(DS_spam["Clase"]>=0)

In [ ]:
# 2.7 Validacion
DS_spam_2.groupBy('clase').count().show(10,False)

In [ ]:
# Revision de los valores
DS_spam_2.show(5, truncate = False)

In [ ]:
#Creando la columna distancia
from pyspark.sql.functions import length,rand 
DS_spam_2=DS_spam_2.withColumn('length',length(DS_spam['texto'])) 
DS_spam_2.orderBy(rand()).show(10,False)

In [ ]:
# Cantidad de texto
DS_spam_2.groupBy('clase').agg({'Length':'mean'}).show()

In [ ]:
DS_spam_2.groupBy('clase').agg({'Length':'median'}).show()

In [ ]:
# Revision preliminar de los comentarios
from pyspark.sql.functions import rand
DS_spam_2.orderBy(rand()).show(10,False)

# 2.  Realizando la codificación de los emaling

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
tokenization=Tokenizer(inputCol='texto',outputCol='tokens')
tokenized_df=tokenization.transform(DS_spam_2)
stopword_removal=StopWordsRemover(inputCol='tokens', outputCol='refined_tokens')
refined_text_df=stopword_removal.transform(tokenized_df)

In [ ]:
refined_text_df.orderBy(rand()).show(10,False)

In [ ]:
# Conteo de las palabras

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
len_udf = udf(lambda s: len(s), IntegerType())
refined_text_df = refined_text_df.withColumn("token_count",
len_udf(col('refined_tokens')))
refined_text_df.orderBy(rand()).show(10)

# 4.  Preparando la información para el modelamiento

In [ ]:
# Creando las variables nevesarias para el modelamiento
from pyspark.ml.feature import CountVectorizer

count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')
cv_text_df=count_vec.fit(refined_text_df).transform(refined_text_df)
cv_text_df.select(['refined_tokens','token_count','features','Clase']).show(10)

In [ ]:
# Variables propias para el modelamiento

model_text_df=cv_text_df.select(['features','token_count','Clase'])

In [ ]:
# Consolidadno toda la información codificada del mensaje en los correos

from pyspark.ml.feature import VectorAssembler
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(model_text_df)
model_text_df.printSchema()

In [ ]:
# Adecuando el formato de la variable Clase a numerica
from pyspark.sql.types import IntegerType
model_text_df = model_text_df.withColumn("Clase", model_text_df["Clase"].cast(IntegerType()))

In [ ]:
model_text_df.printSchema()

# 5.  Seleccionando la muestra de aprendizaje y validacion


In [ ]:
training_df,test_df=model_text_df.randomSplit([0.75,0.25],seed=1234)

In [ ]:
training_df.groupBy('Clase').count().show()

In [ ]:
test_df.groupBy('Clase').count().show()

# 6.  Modelamiento de predictivo de texto

In [ ]:
from pyspark.ml.classification import LogisticRegression

log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Clase').fit(training_df)

In [ ]:
results=log_reg.evaluate(test_df).predictions

In [ ]:
results.show(10,truncate=False)

# 7. Evaluacion del modelo


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

trainingSummary = log_reg.summary
roc = trainingSummary.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))